# DCNN test

DCNN test notebook is a place for exploring customized layers of dcnn with tensorflow.

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.python import debug as tf_debug

*Do Experinemts on the padding function on tensorflow*

In [2]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])

# Set up the padding values
# [1, 1]: Add 1 pad for "height(row)" dimension
# [2, 2]: Add 2 pads for "weight(col)" dimension
paddings = tf.constant([[1, 1,], [2, 2]])

t_pad = tf.pad(t, paddings, "CONSTANT")

with tf.Session() as sess:
    
    r = sess.run(t_pad)
    
    print(r)

[[0 0 0 0 0 0 0]
 [0 0 1 2 3 0 0]
 [0 0 4 5 6 0 0]
 [0 0 0 0 0 0 0]]


In [3]:
# Set up the padding values asymmetric 
# [1, 2]: Prepend 1 pad before the values and append 2 pads after the values in dimension 0
# [3, 4]: Prepend 3 pads befroe the values and append 4 pads after the values in dimension 1
paddings = tf.constant([[1, 2], [3, 4]])

t_pad = tf.pad(t, paddings, "CONSTANT")

with tf.Session() as sess:
    
    print(sess.run(t_pad))


[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 2 3 0 0 0 0]
 [0 0 0 4 5 6 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [4]:
t_4d = tf.constant([[[[1, 2, 3], [4, 5, 6]]]])
paddings = tf.constant([[0,0], [0,0], [1, 1], [2, 2]])

t_pad = tf.pad(t_4d, paddings, "CONSTANT")

with tf.Session() as sess:
    
    r = sess.run(t_pad)
    print(r.shape)
    print(r)

(1, 1, 4, 7)
[[[[0 0 0 0 0 0 0]
   [0 0 1 2 3 0 0]
   [0 0 4 5 6 0 0]
   [0 0 0 0 0 0 0]]]]


## Test: KMaxPooling2D

KmaxPooling is a fundamental layer of dynamic cnn. 

#### Op implementation

In [112]:
import math

# 20180604 Lin, Y.D.: Read how tensorflow does padding dynamically. 
def _kmax_pooling2d(X, pool_size, strides, k, padding):
    
    h = X.shape[0]
    w = X.shape[1]
    
    h_pool = pool_size[0]
    w_pool = pool_size[1]

    h_stride = strides[0]
    w_stride = strides[1]
    
    h_pad = padding[0]
    w_pad = padding[1]
    
    outputs_h = math.ceil((h+h_pad-h_pool+1)/h_stride)
    outputs_w = k * math.ceil((w+w_pad-w_pool+1)/w_stride)
    
    # 20180612 LIN, Y.D. Bugs are still existed.
    outputs = np.zeros((outputs_h, outputs_w), dtype=np.float64)
    
    m, n = 0, 0
    
    for i in range(0, h, h_stride):
        
        for j in range(0, w, w_stride):
            
            flatten_slice = X[i:i+h_pool, j:j+w_pool].ravel()
            top_indices = np.sort(np.argpartition(flatten_slice, -1 * np.arange(k))[-1 * np.arange(k)])
            outputs[m, n:n+k] = flatten_slice[top_indices]
            
            print('top_indices')
            print(top_indices)
            
            n+=k
            
        m+=1
#         top_ks = np.array([], dtype=np.float64)
#         stop = False
        
#         for j in range(0, row, strides):
#             if j + pool_size > row:
                
#                 # 20180531 LIN, Y.D. 
#                 d = row - j if row -j < k else k
#                 top_k_ind = np.argsort(np.argpartition(-1*X[j: row, i], np.arange(d))[:d])
#                 top_k = X[j + top_k_ind, i]

#                 stop = True
                
#             else:
#                 top_k_ind = np.argsort(np.argpartition(-1*X[j: row, i], np.arange(k))[:k])
#                 top_k = X[j + top_k_ind, i]
                
    
#             top_ks = np.concatenate((top_ks, top_k))
            
#             if stop:
#                 break
            
#         kmax_pool.append(top_ks)
        
#         i += 1
    

#     kmax_pool = np.swapaxes(np.stack(kmax_pool), 1, 0)
    return outputs

#### Layer Implementation

In [113]:
#20180607 LIN, Y.D. 
class KMaxPooling2D(tf.layers.Layer):
    
    def __init__(self, pool_size, strides, k, padding='valid', name=None, **kwargs):
        
        super(KMaxPooling2D, self).__init__(name=name, **kwargs)
        
        if padding != 'same' and padding != 'valid':
            raise ValueError('Invalid Padding Options. Only SAME and VALID are valid.')
            
        # 20180610 LIN, Y.D. Add the exception when 
        if strides[0] == 0 and strides[1] == 0:
            raise ValueError('Invalid Stride Values: At least 1 stride value should be greater than 0.')

        self.k = k
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides
    
    def call(self, inputs):
        
        inputs_shape = inputs.get_shape().as_list()
        h = inputs_shape[0]
        w = inputs_shape[1]
        
        h_stride = self.strides[0]
        w_stride = self.strides[1]
        
        h_pool_s = self.pool_size[0]
        w_pool_s = self.pool_size[1]
        
        # 20180612 LIN, Y.D. 
        pool_size = tf.constant(self.pool_size, dtype=tf.int32)
        strides   = tf.constant(self.strides, dtype=tf.int32)
        k = tf.constant(self.k, dtype=tf.int32)
        padding = tf.constant([0, 0], dtype=tf.int32)
        
        if self.padding == 'valid':
            if ((h - h_pool_s) / h_stride + 1) % 1 != 0:
                raise ValueError
            if ((w - w_pool_s) / w_stride + 1) % 1 != 0:
                raise ValueError
        
        elif self.padding == 'same':
            
            h_mod = h % h_stride
            w_mod = w % w_stride
        
            if h_mod == 0:
                h_pad = max([h_pool_s - h_stride, 0])
            else:
                h_pad = max([h_pool_s - h_mod, 0])
            
            if w_mod == 0:
                w_pad = max([w_pool_s - w_stride, 0])
            else:
                w_pad = max([w_pool_s - w_mod, 0])
        
            # 20180606 Y.D. Padding input with 0.
            h_pad_mod_2 = h_pad % 2
            w_pad_mod_2 = w_pad % 2
        
            if h_pad_mod_2 == 0:
                h_pad_top, h_pad_bottom = h_pad//2, h_pad//2
            else:
                h_pad_top, h_pad_bottom = h_pad//2, h_pad//2 + h_pad_mod_2
    
            if w_pad_mod_2 == 0:
                w_pad_left, w_pad_right = w_pad//2, w_pad//2
            else:
                w_pad_left, w_pad_right = w_pad//2, w_pad//2 + w_pad_mod_2
        
            pad_vals = tf.constant(
                [[0, 0], [h_pad_top, h_pad_bottom], [w_pad_left, w_pad_right], [0, 0]], 
                dtype=tf.int32)
        
            inputs = tf.pad(inputs, pad_vals, "CONSTANT")
            
            # 20180612 LIN, Y.D. Update inputs
            padding = tf.constant([h_pad_top+h_pad_bottom, w_pad_left+w_pad_right], dtype=tf.int32)
        
        outputs = tf.py_func(_kmax_pooling2d, [inputs, pool_size, strides, k, padding], tf.float64)
        
        return outputs
    
    # 20180610 LIN, Y.D. TODO: Cannot figure out when to trigger it.
#     def compute_output_shape(self, input_shape):
        
#         print('Compute output shape')
        
#         input_shape = tensor_shape.TensorShape(input_shape).as_list()
#         h = input_shape[1]
#         w = input_shape[2]
        
#         h_stride = self.strides[1]
#         w_stride = self.strides[2]
        
#         if self.padding == 'valid':
#             if ((h - h_pool_s) / h_stride + 1) % 1 != 0:
#                 raise ValueError
#             if ((w - w_pool_s) / w_stride + 1) % 1 != 0:
#                 raise ValueError
        
#         elif self.padding == 'same':
            
#             h_mod = h % h_stride
#             w_mod = w % w_stride
        
#             if h_mod == 0:
#                 h_pad = max([h_pool_s - h_stride, 0])
#             else:
#                 h_pad = max([h_pool_s - h_mod, 0])
            
#             if w_mod == 0:
#                 w_pad = max([w_pool_s - w_stride, 0])
#             else:
#                 w_pad = max([w_pool_s - w_mod, 0])
        
#             # 20180606 Y.D. Padding input with 0.
#             h_pad_mod_2 = h_pad % 2
#             w_pad_mod_2 = w_pad % 2
        
#             if h_pad_mod_2 == 0:
#                 h_pad_top, h_pad_bottom = h_pad//2, h_pad//2
#             else:
#                 h_pad_top, h_pad_bottom = h_pad//2, h_pad//2 + h_pad_mod_2
    
#             if w_pad_mod_2 == 0:
#                 w_pad_left, w_pad_right = w_pad//2, w_pad//2
#             else:
#                 w_pad_left, w_pad_right = w_pad//2, w_pad//2 + w_pad_mod_2
        
#         return tensor_shape.TensorShape(
#           [input_shape[0], h+h_pad, w+w_pad, input_shape[3]])
        

In [114]:
def kmax_pooling2d(inputs, pool_size, strides, k, padding, name=None):
    return KMaxPooling2D(pool_size, strides, k, padding=padding, name=name).apply(inputs)

In [116]:
# 20180605 LIN, Y.D.
with tf.Session() as sess:
    
    # Try if layer works as we all image
    test_x = np.array([[[1,1,3], [3,1,2]]], dtype=np.float64)
    test_input = tf.placeholder(tf.float64, (1, 2, 3))
    test_reshape = tf.reshape(test_input, shape=(1,2,3,1))
    
    # Test Case 1: 
    test_layer = kmax_pooling2d(test_reshape, [2, 2], [1, 1], 2, 'same')
    res = sess.run(test_layer, feed_dict={test_input: test_x })
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 2: Test if exception work.
#     test_layer_2 = kmax_pooling2d(test_reshape, [3, 3], [0, 0], 2, 'same')
#     res = sess.run(test_layer_2, feed_dict={test_input: test_x })
#     print(res)
#     print(res.shape)
#     print('-' * 10)
    
    # Test Case 3:
#     test_x_3 = np.array([[[1, 2, 3], [3, 1, 2], [2, 1, 3]]], dtype=np.float64)
#     test_input_3 = tf.placeholder(tf.float64, (1, 2, 3))
#     test_layer_3 = kmax_pooling2d(test_input_3, [2, 2], [1, 1], 2, 'valid')
#     res = sess.run(test_layer_3, feed_dict={ test_input_3: test_x })
#     print(res)
#     print(res.shape)
    
    
    



top_indices
[3 4]
top_indices
[0 3]
top_indices
[0 3]
[[0. 3. 3. 0. 0. 0.]]
(1, 6)
----------


In [100]:
test_array = np.array([3,2,1,5,6,7,0,2])
print(test_array[np.argpartition(test_array, [-1,-2])])
print(test_array[np.argpartition(test_array, [-1,-2])][-1 * np.arange(1, 3)])
print(np.argpartition(test_array, [-1,-2])[-1 * np.arange(1, 3)])
print(test_array[np.sort(np.argpartition(test_array, [-1,-2])[-1 * np.arange(1, 3)])])
# print(np.argsort(np.argpartition(test_array, [-1,-2])[-1 * np.arange(1, 3)]))
# print(test_array[np.argsort(np.argpartition(test_array, [-1,-2]))])

[0 2 1 2 3 5 6 7]
[7 6]
[5 4]
[6 7]


In [31]:
with tf.Session() as sess:
    test_x = np.array([[[1,1,3], [3,1,2]]], dtype=np.float64)
    test_input = tf.placeholder(tf.float64, (1, 2, 3))
    test_reshape = tf.reshape(test_input, shape=(1,2,3,1))
    pool2d = tf.layers.max_pooling2d(test_reshape, [5, 5], [1, 1])
    
    res = sess.run(pool2d, feed_dict={ test_input: test_x})
    print(res)

ValueError: Negative dimension size caused by subtracting 5 from 2 for 'max_pooling2d_3/MaxPool' (op: 'MaxPool') with input shapes: [1,2,3,1].

In [16]:
print('test')

test


In [ ]:
# 20180609 LIN, Y.D. Test space_to_batch
test_x_1 = np.array([[[[1,1,3], [3,1,2]]]], dtype=np.float64)
test_x_1_placeholder = tf.placeholder(tf.float64, (1, 2, 3, 1))
padding_values = tf.constant([[2,2], [1,1]], dtype=tf.int32)
space_to_batch = tf.space_to_batch(test_x_1_placeholder, padding_values, )

## Test: Folding

Folding is introduced before the last KMax layer.

## Test: Varied Length Input

Varied length of text as input.


## Test: Stack a "non-deep" dcnn (1)

We stack a shallow dcnn to make sure no bug exists in inference stage.


## Test: Stack a "non-deep" dcnn (2)

Train a "non-deep" dcnn with varied length as input.